# 0. Install the dependencies

In [1]:
!pip install open3d
!pip install learning3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!python --version

Python 3.10.13


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import open3d as o3d
import torch
from learning3d.models import PointNet, DGCNN, PPFNet, DCP
from learning3d.models import Classifier

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
device = o3d.core.Device("CUDA:1")
dtype = o3d.core.float32

# 1. Load and preprocess data

In [5]:
"""%%time
mesh = o3d.io.read_triangle_mesh("/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/database/0.stl")
pcd = mesh.sample_points_uniformly(1024)
point_arr = np.asarray(pcd.points)
print(f"Point cloud arr shape {point_arr.shape}, max: {np.max(point_arr)}, min: {np.min(point_arr)}")"""

'%%time\nmesh = o3d.io.read_triangle_mesh("/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/database/0.stl")\npcd = mesh.sample_points_uniformly(1024)\npoint_arr = np.asarray(pcd.points)\nprint(f"Point cloud arr shape {point_arr.shape}, max: {np.max(point_arr)}, min: {np.min(point_arr)}")'

In [6]:
df = pd.read_csv('/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/labels.csv')

In [7]:
df.head()

,query,label
0,0.png,"0.stl,17.stl,70.stl,110.stl,130.stl"
1,1.png,"1.stl,13.stl,19.stl,223.stl,119.stl"
2,10.png,"10.stl,155.stl,237.stl,75.stl,15.stl"
3,100.png,"100.stl,72.stl,216.stl,127.stl,55.stl"
4,101.png,"101.stl,134.stl,248.stl,31.stl,199.stl"


In [8]:
df['label'] = df['label'].apply(lambda x: x.split(','))

In [9]:
# split the df into train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 136 to 102
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   query   201 non-null    object
 1   label   201 non-null    object
dtypes: object(2)
memory usage: 4.7+ KB


In [11]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 165 to 38
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   query   51 non-null     object
 1   label   51 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [12]:
%%time

# Load all the database into the RAM, in form of a dictionary

db_names = os.listdir('/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/database')
#db_list = ['/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/database/' + path for path in db_names]
database_arr = {}

for path in db_names:
    mesh = o3d.io.read_triangle_mesh('/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/database/' + path)
    pcd = mesh.sample_points_uniformly(1024)
    point_arr = np.asarray(pcd.points)
    database_arr[path] = point_arr

CPU times: user 1min 15s, sys: 11.9 s, total: 1min 27s
Wall time: 1min 37s


In [13]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

from timm.data import create_transform
import timm

In [14]:
class MCDataset(Dataset):
    def __init__(self, data, transform : transforms.Compose = None):
        # Get a transform pipeline for image.    
        if transform: 
             self.transform = transform 
        else: 
            self.transform = A.Compose([ 
                A.Resize(224, 224), 
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
                ToTensorV2()
                ], 
                    keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))  
        self.data = data

    def __getitem__(self, index):
        root = '/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/'
        # Anchor image
        img_path = root + 'queries/' + self.data.iloc[index].query
        img = Image.open(img_path).convert("RGB")
        img_tensor = self.transform(img)
        # Positive 3d model
        label_id = self.data.iloc[index].label[0]

        # Negative 3d model (randomly select from the rest of the dataset)
        neg_index = np.random.choice(np.delete(np.arange(len(self.data)), index))
        neg_label_id = self.data.iloc[neg_index].label[0]
        
        return img_tensor, database_arr[label_id], database_arr[neg_label_id]

    def __len__(self):
        return len(self.data)

In [15]:
train_transform = create_transform(224, is_training=True)
test_transform = create_transform(224)

train_set = MCDataset(train_df, train_transform)
test_set = MCDataset(test_df, test_transform)

BATCH_SIZE = 32
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [16]:
for img_tensor, label, neg in train_dataloader:
    print(img_tensor.shape, label.shape, neg.shape)
    break
for img_tensor, label, neg in test_dataloader:
    print(img_tensor.shape, label.shape, neg.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32, 1024, 3]) torch.Size([32, 1024, 3])
torch.Size([32, 3, 224, 224]) torch.Size([32, 1024, 3]) torch.Size([32, 1024, 3])


# 2. Build Model

In [17]:
class MCClassifier(nn.Module):

    def __init__(self, pc_model, image_model, num_pc_features, num_img_features, dropout=0.5):

        super(MCClassifier, self).__init__()

        self.img_model = image_model
        self.pc_model = pc_model
        self.dropout = nn.Dropout(dropout)
        self.batch_norm = nn.BatchNorm1d(512)  # Add batch normalization layer
        self.relu = nn.ReLU()

        # Image MLP
        self.lni1 = nn.Linear(num_img_features, 512)
        self.lni2 = nn.Linear(512,256)
        #self.img_ln = nn.Linear(1280, 1024)
        
        # Point Cloud MLP
        self.lnp1 = nn.Linear(1024, 512)
        self.lnp2 = nn.Linear(512,256)
     
        
        # For scaling the point cloud features masp
        self.conv1d = nn.Conv1d(1024, 1, kernel_size=1)
    def forward(self, image, pc):
        # 2D Image
        image_features = self.img_model(image)
        img_emb = self.dropout(image_features)
        img_emb = self.lni1(img_emb)
        img_emb = self.batch_norm(img_emb)
        img_emb = self.relu(img_emb)
        #img_emb = self.lni2(img_emb)
        #img_emb = self.relu(img_emb)
        
        # Point Cloud
        pc_features = self.pc_model(pc)
        pc_features = self.conv1d(pc_features).squeeze(1)
        pc_emb = self.dropout(pc_features)
        pc_emb = self.lnp1(pc_emb)
        pc_emb = self.batch_norm(pc_emb)  # Apply batch normalization
        pc_emb = self.relu(pc_emb)
        #pc_emb = self.linear2(out)
        #pc_emb = self.relu(out)
        

        return img_emb, pc_emb

In [18]:
"""dgcnn = DGCNN(emb_dims=1024, input_shape='bnc')
pnt_cls = Classifier(feature_model=dgcnn)"""

"dgcnn = DGCNN(emb_dims=1024, input_shape='bnc')\npnt_cls = Classifier(feature_model=dgcnn)"

In [19]:
pn = PointNet(emb_dims=1024, use_bn=True)
pnt_cls = Classifier(feature_model=pn)
pt_path = '/kaggle/input/3d-cls-pretrained-model/pytorch/modelnet40-pt/1/best_model.t7'
pnt_cls.load_state_dict(torch.load(pt_path, map_location='cpu'))
pc_model = pnt_cls.feature_model
pc_model.to('cuda')

PointNet(
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (conv4): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (conv5): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
  (relu): ReLU()
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [20]:
img_model = timm.create_model('mobilenetv2_100', pretrained=True, num_classes = 0).to('cuda')#.get_classifier()
img_model.to('cuda')
print("CU")

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

CU


In [21]:
model = MCClassifier(pc_model, img_model, 1024 ,img_model.num_features).to('cuda')

In [22]:
from torch.optim import Adam
from tqdm import tqdm
loss = nn.TripletMarginLoss(margin=1.0)
optimizer = Adam(model.parameters(), lr= 1e-4)

In [23]:
"""pnt_tensor = torch.from_numpy(point_arr).unsqueeze(0).float().to('cuda')
pnt_tensor.shape

test_transform = create_transform(224)

img = Image.open('/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/queries/0.png')
img_tensor = test_transform(img)
img_tensor.shape

model.eval()
print("CI")

for data in train_dataloader:
    model(anchor_img.to('cuda').float(), pos_char.to('cuda').float())
    
    break"""

'pnt_tensor = torch.from_numpy(point_arr).unsqueeze(0).float().to(\'cuda\')\npnt_tensor.shape\n\ntest_transform = create_transform(224)\n\nimg = Image.open(\'/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/queries/0.png\')\nimg_tensor = test_transform(img)\nimg_tensor.shape\n\nmodel.eval()\nprint("CI")\n\nfor data in train_dataloader:\n    model(anchor_img.to(\'cuda\').float(), pos_char.to(\'cuda\').float())\n    \n    break'

# 3. Training 

In [24]:
def train(model, train_data, val_data, learning_rate, epochs, batch_size):

    train_transform = create_transform(224, is_training=True)
    test_transform = create_transform(224)
    train_set = MCDataset(train_df, train_transform)
    test_set = MCDataset(test_df, test_transform)
    BATCH_SIZE = batch_size
    train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)
    val_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.TripletMarginLoss(margin=1.0)
    optimizer = Adam(model.parameters(), lr= learning_rate)
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    
    for epoch_num in range(epochs):

        train_loss = []
        mrr_train = []
        #model.train()
        for data in tqdm(train_dataloader):
            anchor_img, pos_char, neg_char = data
            anchor_feat, pos_feat = model(anchor_img.to('cuda').float(), pos_char.to('cuda').float())
            _, neg_feat = model(anchor_img.to('cuda').float(), neg_char.to('cuda').float())

            # Calculate triplet loss
            batch_loss = criterion(anchor_feat, pos_feat, neg_feat)
            train_loss.append(float(batch_loss))

            # Backpropagation and update weights
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
            
        avg_loss = np.mean(train_loss)
        print("epoch %2d train end : avg_loss = %.4f" % (epoch_num+1, avg_loss))
        #print("epoch %2d train end : avg_acc = %.4f" % (epoch_num+1, avg_train_acc))

        test_loss = []
        test_mrr = []
        #model.eval()
        with torch.no_grad():

            for data in tqdm(val_dataloader):

                anchor_img, pos_char, neg_char = data
                anchor_feat, pos_feat = model(anchor_img.to('cuda').float(), pos_char.to('cuda').float())
                _, neg_feat = model(anchor_img.to('cuda').float(), neg_char.to('cuda').float())

                batch_loss = criterion(anchor_feat, pos_feat, neg_feat)
                test_loss.append(float(batch_loss))

                #batch_acc = (output.round() == val_label).float().mean()
                #test_acc.append(float(batch_acc))

            avg_test_loss = np.mean(test_loss)
            #mrr_test = np.mean(test_acc)
            print("epoch %2d test end : avg_loss = %.4f" % (epoch_num+1, avg_test_loss))
            #print("epoch %2d test end : avg_acc = %.4f" % (epoch_num+1, avg_test_acc))
            
    torch.save(model, 'retrival_net.ckpt')


In [25]:
batch_size = 16
EPOCHS = 100
LR = 1e-3
# SAVE TRAINING LOGS.
train(model, train_df, test_df, LR, EPOCHS, batch_size)

100%|██████████| 13/13 [00:04<00:00,  2.63it/s]


epoch  1 train end : avg_loss = 1.2226


100%|██████████| 4/4 [00:00<00:00,  5.19it/s]


epoch  1 test end : avg_loss = 1.2687


100%|██████████| 13/13 [00:03<00:00,  4.19it/s]


...

epoch 99 test end : avg_loss = 0.8942


100%|██████████| 13/13 [00:03<00:00,  4.05it/s]


epoch 100 train end : avg_loss = 0.1546


100%|██████████| 4/4 [00:00<00:00,  5.96it/s]

epoch 100 test end : avg_loss = 1.2214


# 4. Evaluation

In [26]:
test_df.head()

,query,label
165,185.png,"[185.stl, 85.stl, 200.stl, 214.stl, 115.stl]"
6,103.png,"[103.stl, 19.stl, 218.stl, 141.stl, 214.stl]"
111,59.png,"[59.stl, 109.stl, 116.stl, 133.stl, 222.stl]"
172,191.png,"[191.stl, 135.stl, 81.stl, 100.stl, 118.stl]"
115,138.png,"[138.stl, 60.stl, 148.stl, 235.stl, 226.stl]"


In [27]:
model.eval()
mrr_test = []
for q in test_df['query']:
    
    # Load the query image
    img = Image.open('/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/queries/' + q).convert("RGB")
    img_tensor = test_transform(img).unsqueeze(0).to('cuda').float()

    label = test_df[test_df['query'] == q].label.values[0][0]
    print(f"Query with image: {q}, label: {label}")
    # Calculate the distance between the query image and all the database images
    distances = {}
    for db in db_names:
        db_feat = torch.from_numpy(database_arr[db]).unsqueeze(0).to('cuda').float()
        img_emb, db_emb = model(img_tensor, db_feat)
        dist = torch.nn.functional.pairwise_distance(img_emb, db_emb)
        distances[db] = dist.item()

    # Sort the distances
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    # Calculate the MRR
    mrr = 0
    for i, (db, dist) in enumerate(sorted_distances):
        if db == label:
            print(f"Matching at pos {i}")
            mrr = 1/(i+1)
            break
    mrr_test.append(mrr)
    print(f"Query: {q}, MRR@5: {mrr}")
print(f"Test MRR {sum(mrr_test)/ len(mrr_test)}")

Query with image: 185.png, label: 185.stl
Matching at pos 35
Query: 185.png, MRR@5: 0.027777777777777776
Query with image: 103.png, label: 103.stl
Matching at pos 22
Query: 103.png, MRR@5: 0.043478260869565216
Query with image: 59.png, label: 59.stl
Matching at pos 161
Query: 59.png, MRR@5: 0.006172839506172839
Query with image: 191.png, label: 191.stl
Matching at pos 116
Query: 191.png, MRR@5: 0.008547008547008548
Query with image: 138.png, label: 138.stl
Matching at pos 179
Query: 138.png, MRR@5: 0.005555555555555556
Query with image: 201.png, label: 201.stl
Matching at pos 110
Query: 201.png, MRR@5: 0.009009009009009009
Query with image: 217.png, label: 217.stl
Matching at pos 8
Query: 217.png, MRR@5: 0.1111111111111111
Query with image: 250.png, label: 250.stl
Matching at pos 63
Query: 250.png, MRR@5: 0.015625
Query with image: 106.png, label: 106.stl
Matching at pos 18
Query: 106.png, MRR@5: 0.05263157894736842
Query with image: 126.png, label: 126.stl
Matching at pos 131
Query: 1